检测是否有MNIST数据集：

In [1]:
from torchvision import datasets, transforms

train_set = datasets.MNIST("data",train=True,download=True, transform=transforms.ToTensor(),)
test_set = datasets.MNIST("data",train=False,download=True, transform=transforms.ToTensor(),)


In [2]:
import os
import matplotlib.pyplot as plt
import torch
from PIL import Image
from torch import nn
from torch.nn import Conv2d, Linear, ReLU
from torch.nn import MaxPool2d
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader


# Dataset:创建数据集的函数；__init__:初始化数据内容和标签
# __geyitem:获取数据内容和标签
# __len__:获取数据集大小
# daataloader:数据加载类，接受来自dataset已经加载好的数据集
# torchbision:图形库，包含预训练模型，加载数据的函数、图片变换，裁剪、旋转等
# torchtext:处理文本的工具包，将不同类型的额文件转换为datasets

# 预处理：将两个步骤整合在一起
transform = transforms.Compose({
    transforms.ToTensor(),  # 将灰度图片像素值（0~255）转为Tensor（0~1），方便后续处理
   })


检测一下是否能加载数据

In [3]:
# 加载数据集
# 训练数据集
train_data = MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
# transform：指示加载的数据集应用的数据预处理的规则，shuffle：洗牌，是否打乱输入数据顺序
# 测试数据集
test_data = MNIST(root="./data", train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=True)

train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度：{}".format(train_data_size))
print("测试数据集的长度：{}".format(test_data_size))


训练数据集的长度：60000
测试数据集的长度：10000


在进一步检测一下

In [4]:
print("图像形状:", train_data[420][0].shape)  # 应该是 torch.Size([1, 28, 28])
print("标签:", train_data[420][1])           # 应该是 0-9 的数字

图像形状: torch.Size([1, 28, 28])
标签: 5


开始构建模型！

模型主要由两个卷积层，两个池化层，以及三个全连接层构成，激活函数使用relu

In [5]:
class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        # 定义保持不变
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.maxpool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.maxpool2 = nn.MaxPool2d(2)
        self.linear1 = nn.Linear(320, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    # --- 新增：前半段 (特征提取) ---
    def features(self, x):
        x = self.relu(self.maxpool1(self.conv1(x)))
        x = self.relu(self.maxpool2(self.conv2(x)))
        # 这里输出的是 [Batch, 20, 4, 4] 的特征图
        return x

    # --- 新增：后半段 (分类器) ---
    def classifier(self, x):
        # 接收特征图，先展平
        x = x.view(x.size(0), -1) 
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        return x

    # 原来的 forward 只是把两段连起来
    def forward(self, x):
        feat = self.features(x)
        out = self.classifier(feat)
        return out

# 损失函数CrossentropyLoss
model = MnistModel()#实例化
criterion = nn.CrossEntropyLoss()   # 交叉熵损失，相当于Softmax+Log+NllLoss
# 线性多分类模型Softmax,给出最终预测值对于10个类别出现的概率，Log:将乘法转换为加法，减少计算量，保证函数的单调性
# NLLLoss:计算损失，此过程不需要手动one-hot编码，NLLLoss会自动完成

# SGD，优化器，梯度下降算法e
optimizer = torch.optim.SGD(model.parameters(), lr=0.14)#lr:学习率


关于index可以使用两种处理方式，这里选择的是利用enumerate，也可以选择手动递加

In [6]:
# 模型训练
def train():
    for index, data in enumerate(train_loader):#获取训练数据以及对应标签
       input, target = data   # input为输入数据，target为标签
       y_predict = model(input) #模型预测
       loss = criterion(y_predict, target)
       optimizer.zero_grad() #梯度清零。每次都清空，确保只计算当前批次的梯度
       loss.backward()#loss值反向传播
       optimizer.step()#更新参数
       if index % 100 == 0: # 每一百次保存一次模型，输出损失值
           torch.save(model.state_dict(), "./model/model.pkl")   # 保存模型
           torch.save(optimizer.state_dict(), "./model/optimizer.pkl")
           print("训练次数为：{}，损失值为：{}".format(index, loss.item() ))




In [7]:
# 加载模型
# 在训练前创建模型保存目录
os.makedirs('./model', exist_ok=True)  # 如果目录不存在就创建

# 然后正常训练，第一次不会加载模型
if os.path.exists('./model/model.pkl') and os.path.exists('./model/optimizer.pkl'):
    model.load_state_dict(torch.load("./model/model.pkl"))
    optimizer.load_state_dict(torch.load("./model/optimizer.pkl"))  # 这行很重要！
    print("✓ 恢复模型和优化器状态")
else:
    print("✓ 开始新的训练")



✓ 恢复模型和优化器状态


C:\Users\MSI\AppData\Local\Temp\ipykernel_11900\2976271716.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./model/model.pkl"))
C:\User

In [8]:
# 模型测试
def test():
    correct = 0     # 正确预测的个数
    total = 0   # 总数
    with torch.no_grad():   # 测试不用计算梯度
        for data in test_loader:
            input, target = data
            output = model(input)   # output输出10个预测取值，概率最大的为预测数
            probability, predict = torch.max(input=output.data, dim=1)    # 返回一个元祖，第一个为最大概率值，第二个为最大概率值的下标
            # loss = criterion(output, target)
            total += target.size(0)  # target是形状为（batch_size,1)的矩阵，使用size（0）取出该批的大小
            correct += (predict == target).sum().item()  # predict 和target均为（batch_size,1)的矩阵，sum求出相等的个数
        print("测试准确率为：%.6f" %(correct / total))


In [13]:
#测试识别函数
if __name__ == '__main__':
    #训练与测试
    for i in range(1):#训练和测试进行15轮
        print("————————第{}轮测试开始——————".format (i + 1))
        train()
        test()


————————第1轮测试开始——————
训练次数为：0，损失值为：0.00010700365965021774
训练次数为：100，损失值为：6.244760879781097e-05
训练次数为：200，损失值为：0.002269541844725609
训练次数为：300，损失值为：0.004874390549957752
训练次数为：400，损失值为：4.483581142267212e-05
训练次数为：500，损失值为：0.0008190689259208739
训练次数为：600，损失值为：5.862650141352788e-06
训练次数为：700，损失值为：0.00012583246279973537
训练次数为：800，损失值为：9.534152195556089e-05
训练次数为：900，损失值为：0.0002775305765680969
测试准确率为：0.991800
